In [1]:
dataset_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno"
yaml_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/train.yaml"

model_path = "/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/process_data/yolov8l_indoorvg4.pt"

import os
from ultralytics import YOLO
import torch

# load model
model = YOLO(model_path)

from ultralytics.cfg import get_cfg

cfg = get_cfg()

# load train data
# load yaml
from ultralytics.utils import yaml_load
from ultralytics.data import build_dataloader, build_yolo_dataset

data = yaml_load(yaml_path)

# loda dataset
from ultralytics.data import YOLODataset

# load dataset
splits = [data['train'], data['val'], data['test']]

split = splits[0]
img_dir = os.path.join(split, 'images')
dataset = build_yolo_dataset(cfg, img_dir, 1, data)

/home/maelic/miniconda3/envs/phd/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load train data
# load yaml
from ultralytics.utils import yaml_load
from ultralytics.data import build_dataloader, build_yolo_dataset

data = yaml_load(yaml_path)

# loda dataset
from ultralytics.data import YOLODataset

# load dataset
splits = [data['train'], data['val'], data['test']]

split = splits[0]
img_dir = os.path.join(split, 'images')
dataset = build_yolo_dataset(cfg, img_dir, 1, data)

print(dataset)


train: Scanning /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/train/labels.cache... 11739 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11739/11739 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


# Utils

In [4]:
new_dataset_path = dataset_path.strip().replace('YOLO_anno', 'YOLO_anno_2')
# create dir
os.makedirs(new_dataset_path, exist_ok=True)

In [6]:
from utils import bbox_iou
from tqdm import tqdm

iou_thres = 0.5
conf_thres = 0.5

dest_anno_path = new_dataset_path + '/train/labels'
# mkdir
if not os.path.exists(dest_anno_path):
    os.makedirs(dest_anno_path)

num_total = 0

# run model on dataset
for i, img in enumerate(tqdm(dataset)):
    # get img name
    img_name = img['im_file'].split('/')[-1].split('.')[0]
    # get anno file
    anno_file = dataset.label_files[i]
    # read anno file
    with open(anno_file, 'r') as f:
        anno = f.readlines()
    # get gt boxes
    gt_boxes = img['bboxes'].cuda()

    # forward pass
    results = model(img['im_file'], verbose=False)

    # iou on gt boxes and predictions
    for i, box in enumerate(results[0].boxes):
        iou = bbox_iou(box.xyxy, gt_boxes, xywh=True)

        if all(iou < iou_thres):
            # the box is a candidate for a new object
            if box.conf > conf_thres:
                # add to anno
                bbox = box.xyxyn[0]
                anno.append(f"{int(box.cls.item())} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
                num_total += 1

    # write out to dest_anno_path
    with open(dest_anno_path + f'/{img_name}.txt', 'w') as f:
        f.writelines(anno)

print(f"Total new objects added: {num_total}")

100%|██████████| 11739/11739 [08:18<00:00, 23.56it/s]

Total new objects added: 41194


In [3]:
# Fine-tune the model with new data

from ultralytics import YOLO

# Train the model
results = model.train(data='/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno_2/train.yaml', epochs=10, batch=16, imgsz=640, project='indoorvg4_benchmark', name='yolov8l_finetune', exist_ok=True)

Ultralytics YOLOv8.1.31 🚀 Python-3.11.5 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16117MiB)
engine/trainer: task=detect, mode=train, model=/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/process_data/yolov8l_indoorvg4.pt, data=/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno_2/train.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=indoorvg4_benchmark, name=yolov8l_finetune, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream

2024-03-29 16:28:19,090	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-29 16:28:19,281	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: maelic. Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


...


val: Scanning /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/val/labels.cache... 2935 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2935/2935 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/val/images/285776.jpg: 1 duplicate labels removed


Plotting labels to indoorvg4_benchmark/yolov8l_finetune/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000114, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to indoorvg4_benchmark/yolov8l_finetune
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10        11G      1.497      1.936      1.819        197        640: 100%|██████████| 734/734 [07:54<00:00,  1.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 92/92 [00:34<00:00,  2.66it/s]


                   all       2935      39720      0.459      0.426        0.4      0.237
